In [ ]:
import pandas as pd
filtered_df = pd.read_csv('/content/drive/My Drive/covid19tweetswithtone.csv')

In [ ]:
filtered_df.dropna(subset=['tone'], inplace=True)

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [ ]:
!sh setup.sh

In [ ]:
!pip install "transformers"
!pip install "simpletransformers"==0.34.4

In [ ]:
!pip install --upgrade wandb -qq

In [ ]:
import wandb
wandb.login()

True

In [ ]:
new_df = filtered_df.drop(filtered_df.columns.difference(['tweet_id','tweet', 'tone']), axis=1)
new_df.head(1)

,tweet,tweet_id,tone
0,GREAT #FridayFeeling 👍 being #confined to our ...,1241237439156563969,"{""document_tone"": {""tones"": [{""score"": 0.94642..."


In [ ]:
# new_df = filtered_df.copy()

In [ ]:
new_df['anger'] = 0
new_df['fear'] = 0
new_df['joy'] = 0
new_df['sadness'] = 0
new_df['analytical'] = 0
new_df['confident'] = 0
new_df['tentative'] = 0
new_df

,tweet,tweet_id,tone,anger,fear,joy,sadness,analytical,confident,tentative
0,GREAT #FridayFeeling 👍 being #confined to our ...,1241237439156563969,"{""document_tone"": {""tones"": [{""score"": 0.94642...",0,0,0,0,0,0,0
1,Goofing around on Photoshop after an afternoon...,1241832474864386049,"{""document_tone"": {""tones"": [{""score"": 0.56807...",0,0,0,0,0,0,0
2,#COVID19 special discount\n\nDuring this f**k...,1241797422717849600,"{""document_tone"": {""tones"": [{""score"": 0.69956...",0,0,0,0,0,0,0
3,Nice #eQSL from @ea3fhp from this morning #con...,1242207479771156484,"{""document_tone"": {""tones"": [{""score"": 0.81180...",0,0,0,0,0,0,0
4,What you guys up to today...I’ve just been on ...,1242138919195619336,"{""document_tone"": {""tones"": [{""score"": 0.54455...",0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
12456,Isolation Day 9. How is your lockdown locking ...,1245050063388377088,"{""document_tone"": {""tones"": []}}",0,0,0,0,0,0,0
12457,Self isolation during the #COVID19 pandemic me...,1245050018878443524,"{""document_tone"": {""tones"": []}, ""sentences_to...",0,0,0,0,0,0,0
12458,"Log:\nWeek 3, day 2 of isolation:\nHave caught...",1245049543340642304,"{""document_tone"": {""tones"": [{""score"": 0.79105...",0,0,0,0,0,0,0
12459,Did Dremo just indirectly Diss M.I Abaga? This...,1245049398750588934,"{""document_tone"": {""tones"": [{""score"": 0.93143...",0,0,0,0,0,0,0


In [ ]:
import json

# Converting tones to multihot encoded vectors
for i, row in new_df.iterrows():
  try:
    tone = json.loads(row.tone)
    tones = tone['document_tone']['tones']
    for tone in tones:
        new_df.at[i,tone['tone_id']] = 1
  except:
    continue
new_df
    

,tweet,tweet_id,tone,anger,fear,joy,sadness,analytical,confident,tentative
0,GREAT #FridayFeeling 👍 being #confined to our ...,1241237439156563969,"{""document_tone"": {""tones"": [{""score"": 0.94642...",0,0,1,0,0,0,0
1,Goofing around on Photoshop after an afternoon...,1241832474864386049,"{""document_tone"": {""tones"": [{""score"": 0.56807...",0,0,1,0,1,0,1
2,#COVID19 special discount\n\nDuring this f**k...,1241797422717849600,"{""document_tone"": {""tones"": [{""score"": 0.69956...",0,0,1,0,0,0,1
3,Nice #eQSL from @ea3fhp from this morning #con...,1242207479771156484,"{""document_tone"": {""tones"": [{""score"": 0.81180...",0,0,1,0,1,0,0
4,What you guys up to today...I’ve just been on ...,1242138919195619336,"{""document_tone"": {""tones"": [{""score"": 0.54455...",0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
12456,Isolation Day 9. How is your lockdown locking ...,1245050063388377088,"{""document_tone"": {""tones"": []}}",0,0,0,0,0,0,0
12457,Self isolation during the #COVID19 pandemic me...,1245050018878443524,"{""document_tone"": {""tones"": []}, ""sentences_to...",0,0,0,0,0,0,0
12458,"Log:\nWeek 3, day 2 of isolation:\nHave caught...",1245049543340642304,"{""document_tone"": {""tones"": [{""score"": 0.79105...",0,0,0,1,0,1,0
12459,Did Dremo just indirectly Diss M.I Abaga? This...,1245049398750588934,"{""document_tone"": {""tones"": [{""score"": 0.93143...",0,0,0,0,0,0,1


In [ ]:
print(new_df['anger'].value_counts())
print(new_df['analytical'].value_counts())
print(new_df['tentative'].value_counts())
print(new_df['confident'].value_counts())
print(new_df['sadness'].value_counts())
print(new_df['fear'].value_counts())
print(new_df['joy'].value_counts())

0    12191
1      268
Name: anger, dtype: int64
0    8095
1    4364
Name: analytical, dtype: int64
0    9323
1    3136
Name: tentative, dtype: int64
0    10086
1     2373
Name: confident, dtype: int64
0    11054
1     1405
Name: sadness, dtype: int64
0    12172
1      287
Name: fear, dtype: int64
0    8185
1    4274
Name: joy, dtype: int64


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)
anger_df = new_df.drop(new_df.columns.difference(['anger','created']), axis=1)
anger_df = anger_df.groupby(["created"]).anger.sum().reset_index()
plt.plot(anger_df['created'], anger_df['anger'], label='anger', color='red')
SaleTransaction.query(ancestor=user.key).filter(ndb.OR(
analytical_df = new_df.drop(new_df.columns.difference(['analytical','created']), axis=1)
analytical_df = analytical_df.groupby(["created"]).analytical.sum().reset_index()
plt.plot(analytical_df['created'], analytical_df['analytical'], label='analytical', color='blue')

tentative = new_df.drop(new_df.columns.difference(['tentative','created']), axis=1)
tentative = tentative.groupby(["created"]).tentative.sum().reset_index()
plt.plot(tentative['created'], tentative['tentative'], label='tentative', color='yellow')

sadness = new_df.drop(new_df.columns.difference(['sadness','created']), axis=1)
sadness = sadness.groupby(["created"]).sadness.sum().reset_index()
plt.plot(sadness['created'], sadness['sadness'], label='sadness', color='purple')

fear = new_df.drop(new_df.columns.difference(['fear','created']), axis=1)
fear = fear.groupby(["created"]).fear.sum().reset_index()
plt.plot(fear['created'], fear['fear'], label='fear', color='black')

joy = new_df.drop(new_df.columns.difference(['joy','created']), axis=1)
joy = joy.groupby(["created"]).joy.sum().reset_index()
plt.plot(joy['created'], joy['joy'], label='joy', color='orange')


confident = new_df.drop(new_df.columns.difference(['confident','created']), axis=1)
confident = confident.groupby(["created"]).confident.sum().reset_index()
plt.plot(confident['created'], confident['confident'], label='confident', color='green')


plt.legend()
plt.show()

KeyError: ignored

In [ ]:
new_df['labels'] = list(zip(new_df.anger.tolist(), new_df.fear.tolist(), new_df.joy.tolist(), new_df.sadness.tolist(),  new_df.analytical.tolist(), new_df.confident.tolist(), new_df.tentative.tolist()))
new_df['text'] = new_df['tweet'].apply(lambda x: x.replace('\n', ' '))

new_df

,tweet,tweet_id,tone,anger,fear,joy,sadness,analytical,confident,tentative,labels,text
0,GREAT #FridayFeeling 👍 being #confined to our ...,1241237439156563969,"{""document_tone"": {""tones"": [{""score"": 0.94642...",0,0,1,0,0,0,0,"(0, 0, 1, 0, 0, 0, 0)",GREAT #FridayFeeling 👍 being #confined to our ...
1,Goofing around on Photoshop after an afternoon...,1241832474864386049,"{""document_tone"": {""tones"": [{""score"": 0.56807...",0,0,1,0,1,0,1,"(0, 0, 1, 0, 1, 0, 1)",Goofing around on Photoshop after an afternoon...
2,#COVID19 special discount\n\nDuring this f**k...,1241797422717849600,"{""document_tone"": {""tones"": [{""score"": 0.69956...",0,0,1,0,0,0,1,"(0, 0, 1, 0, 0, 0, 1)",#COVID19 special discount During this f**kin...
3,Nice #eQSL from @ea3fhp from this morning #con...,1242207479771156484,"{""document_tone"": {""tones"": [{""score"": 0.81180...",0,0,1,0,1,0,0,"(0, 0, 1, 0, 1, 0, 0)",Nice #eQSL from @ea3fhp from this morning #con...
4,What you guys up to today...I’ve just been on ...,1242138919195619336,"{""document_tone"": {""tones"": [{""score"": 0.54455...",0,0,0,1,0,0,1,"(0, 0, 0, 1, 0, 0, 1)",What you guys up to today...I’ve just been on ...
...,...,...,...,...,...,...,...,...,...,...,...,...
12456,Isolation Day 9. How is your lockdown locking ...,1245050063388377088,"{""document_tone"": {""tones"": []}}",0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0)",Isolation Day 9. How is your lockdown locking ...
12457,Self isolation during the #COVID19 pandemic me...,1245050018878443524,"{""document_tone"": {""tones"": []}, ""sentences_to...",0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0, 0)",Self isolation during the #COVID19 pandemic me...
12458,"Log:\nWeek 3, day 2 of isolation:\nHave caught...",1245049543340642304,"{""document_tone"": {""tones"": [{""score"": 0.79105...",0,0,0,1,0,1,0,"(0, 0, 0, 1, 0, 1, 0)","Log: Week 3, day 2 of isolation: Have caught u..."
12459,Did Dremo just indirectly Diss M.I Abaga? This...,1245049398750588934,"{""document_tone"": {""tones"": [{""score"": 0.93143...",0,0,0,0,0,0,1,"(0, 0, 0, 0, 0, 0, 1)",Did Dremo just indirectly Diss M.I Abaga? This...


In [ ]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(new_df, test_size=0.2)

## BERT

In [ ]:
import wandb
from simpletransformers.classification import MultiLabelClassificationModel
model = MultiLabelClassificationModel('bert', 'bert-base-cased', num_labels=7, args={'evaluate_during_training': True, 'wandb_project': 'covid19tweets', 'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 250, 'output_dir':'/content/models0'})

In [ ]:
model.train_model(train_df, eval_df=eval_df)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.811060

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.820361

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.335796

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.082901Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1048576.0
Running loss: 0.204388Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1048576.0
Running loss: 0.274051


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
result

{'LRAP': 0.9216673513974878, 'eval_loss': 0.23894647028870308}

In [ ]:
# !git clone https://github.com/huggingface/transformers
# !cd transformers
!pip install /content/transformers/

Processing ./transformers
  Using cached https://files.pythonhosted.org/packages/e8/bd/e5abec46af977c8a1375c1dca7cb1e5b3ec392ef279067af7f6bc50491a0/tokenizers-0.8.0rc4-cp36-cp36m-manylinux1_x86_64.whl
  Created wheel for transformers: filename=transformers-3.0.1-cp36-none-any.whl size=757375 sha256=2eef782b08afb896545a9ea6f19ece0108d08eaa3294283b3bd4c46a65f75ba6
  Stored in directory: /tmp/pip-ephem-wheel-cache-koqe97n0/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
  Found existing installation: transformers 3.0.1
    Uninstalling transformers-3.0.1:
      Successfully uninstalled transformers-3.0.1


## Roberta

In [ ]:
train_df = train_df[['text', 'labels']]
eval_df = eval_df[['text', 'labels']]

In [ ]:
eval_df

NameError: ignored

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel
model_roberta = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=7, args={'evaluate_during_training': True, 'wandb_project': 'covid19tweets', 'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 250, 'output_dir':'/content/models2'})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

In [ ]:
import pickle
model_roberta.train_model(train_df, eval_df=eval_df)



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.635643

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.349342

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.245222Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.188426

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.164146Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 262144.0
Running loss: 0.516222Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
Running loss: 0.053215


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
result, model_outputs, wrong_predictions = model_roberta.eval_model(eval_df)

In [ ]:
result

{'LRAP': 0.9266943106830746, 'eval_loss': 0.22536635711693612}

In [ ]:
model_outputs

array([[0.01757812, 0.01924133, 0.98876953, ..., 0.24108887, 0.03482056,
        0.5996094 ],
       [0.02763367, 0.02958679, 0.6074219 , ..., 0.4716797 , 0.98339844,
        0.01678467],
       [0.00985718, 0.02064514, 0.08361816, ..., 0.93603516, 0.03372192,
        0.09008789],
       ...,
       [0.01020813, 0.01312256, 0.84765625, ..., 0.04263306, 0.0164032 ,
        0.0171051 ],
       [0.00374985, 0.00651169, 0.13989258, ..., 0.2368164 , 0.00993347,
        0.35620117],
       [0.00945282, 0.01438904, 0.87890625, ..., 0.66552734, 0.0241394 ,
        0.9082031 ]], dtype=float32)

In [ ]:
wrong_predictions

In [ ]:
!cp /content/models/pytorch_model.bin '/content/drive/My Drive/bert.bin'

In [ ]:
!cp /content/models/config.json '/content/drive/My Drive/bertconfig.json'

In [ ]:
!cp /content/models2/pytorch_model.bin '/content/drive/My Drive/roberta.bin'

In [ ]:
!cp /content/models2/config.json '/content/drive/My Drive/robertaconfig.json'

## Electra

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel
model_electra = MultiLabelClassificationModel('electra', 'google/electra-small-discriminator', num_labels=7, args={'evaluate_during_training': True, 'wandb_project': 'covid19tweets', 'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 250, 'output_dir':'/content/models3'})

In [ ]:
model_electra.train_model(train_df, eval_df=eval_df)



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.670304

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.571963

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.240651Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 524288.0
Running loss: 0.328702

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.314781Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 524288.0
Running loss: 0.419402Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 524288.0
Running loss: 0.564566


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
!cp /content/models3/pytorch_model.bin '/content/drive/My Drive/electra.bin'
!cp /content/models3/config.json '/content/drive/My Drive/electraconfig.json'

## XLNet

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel
model_xlnet = MultiLabelClassificationModel('xlnet','xlnet-base-cased', num_labels=7, args={'evaluate_during_training': True, 'wandb_project': 'covid19tweets', 'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 250, 'output_dir':'/content/models4'})

In [ ]:
model_xlnet.train_model(train_df, eval_df=eval_df)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.695595

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.340285

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.049145

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.076072Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 524288.0
Running loss: 0.067973


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [ ]:
!cp /content/models4/pytorch_model.bin '/content/drive/My Drive/xlnet.bin'
!cp /content/models4/config.json '/content/drive/My Drive/xlnetconfig.json'